## Assignment 1/Part - 2A

In [11]:
#Configration 1
#Average Pooling with relu


from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D

import os
import pickle
import numpy as np

batch_size = 32
num_classes = 10
epochs = 22
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model_1P.h5'

# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model_1P = Sequential()

model_1P.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model_1P.add(Activation('relu'))
model_1P.add(Conv2D(32, (3, 3)))
model_1P.add(Activation('relu'))
model_1P.add(AveragePooling2D(pool_size=(2, 2)))
model_1P.add(Dropout(0.25))

model_1P.add(Conv2D(64, (3, 3), padding='same'))
model_1P.add(Activation('relu'))
model_1P.add(Conv2D(64, (3, 3)))
model_1P.add(Activation('relu'))
model_1P.add(AveragePooling2D(pool_size=(2, 2)))
model_1P.add(Dropout(0.25))

model_1P.add(Flatten())
model_1P.add(Dense(512))
model_1P.add(Activation('relu'))
model_1P.add(Dropout(0.5))
model_1P.add(Dense(num_classes))
model_1P.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model_1P.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

if not data_augmentation:
    print('Not using data augmentation.')
    model_1P.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model_1P.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        steps_per_epoch=x_train.shape[0] // batch_size,
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)

# Load label names to use in prediction results
label_list_path = 'datasets/cifar-10-batches-py/batches.meta'


keras_dir = os.path.expanduser(os.path.join('~', '.keras'))
datadir_base = os.path.expanduser(keras_dir)
if not os.access(datadir_base, os.W_OK):
    datadir_base = os.path.join('/tmp', '.keras')
label_list_path = os.path.join(datadir_base, label_list_path)

with open(label_list_path, mode='rb') as f:
    labels = pickle.load(f)

# Evaluate model with test data set and share sample prediction results
evaluation = model_1P.evaluate_generator(datagen.flow(x_test, y_test,
                                                   batch_size=batch_size,
                                                   shuffle=False),
                                      steps=x_test.shape[0] // batch_size,
                                      workers=4)
print('Model Accuracy = %.2f' % (evaluation[1]))

predict_gen = model_1P.predict_generator(datagen.flow(x_test, y_test,
                                                   batch_size=batch_size,
                                                   shuffle=False),
                                      steps=x_test.shape[0] // batch_size,
                                      workers=4)

for predict_index, predicted_y in enumerate(predict_gen):
    actual_label = labels['label_names'][np.argmax(y_test[predict_index])]
    predicted_label = labels['label_names'][np.argmax(predicted_y)]
    print('Actual Label = %s vs. Predicted Label = %s' % (actual_label,
                                                          predicted_label))
    if predict_index == num_predictions:
        break

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
Using real-time data augmentation.
Epoch 1/22
1562/1562 [==============================] - 106s - loss: 1.8580 - acc: 0.3181 - val_loss: 1.6289 - val_acc: 0.4145
Epoch 2/22
1562/1562 [==============================] - 100s - loss: 1.6055 - acc: 0.4127 - val_loss: 1.3994 - val_acc: 0.5008
Epoch 3/22
1562/1562 [==============================] - 104s - loss: 1.4928 - acc: 0.4570 - val_loss: 1.3591 - val_acc: 0.5154
Epoch 4/22
1562/1562 [==============================] - 102s - loss: 1.4194 - acc: 0.4898 - val_loss: 1.2625 - val_acc: 0.5482
Epoch 5/22
1562/1562 [==============================] - 107s - loss: 1.3551 - acc: 0.5156 - val_loss: 1.1952 - val_acc: 0.5729
Epoch 6/22
1562/1562 [==============================] - 102s - loss: 1.3036 - acc: 0.5348 - val_loss: 1.1264 - val_acc: 0.5995
Epoch 7/22
1562/1562 [==============================] - 101s - loss: 1.2571 - acc: 0.5523 - val_loss: 1.1303 - val_acc: 0.6031
Epo

In [12]:
#Configration 2
#Default Setting: Relu with Max Pooling



from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D

import os
import pickle
import numpy as np

batch_size = 32
num_classes = 10
epochs = 22
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model_1A.h5'

# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model_1A = Sequential()

model_1A.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model_1A.add(Activation('tanh'))
model_1A.add(Conv2D(32, (3, 3)))
model_1A.add(Activation('tanh'))
model_1A.add(MaxPooling2D(pool_size=(2, 2)))
model_1A.add(Dropout(0.25))

model_1A.add(Conv2D(64, (3, 3), padding='same'))
model_1A.add(Activation('tanh'))
model_1A.add(Conv2D(64, (3, 3)))
model_1A.add(Activation('tanh'))
model_1A.add(MaxPooling2D(pool_size=(2, 2)))
model_1A.add(Dropout(0.25))

model_1A.add(Flatten())
model_1A.add(Dense(512))
model_1A.add(Activation('tanh'))
model_1A.add(Dropout(0.5))
model_1A.add(Dense(num_classes))
model_1A.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model_1A.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

if not data_augmentation:
    print('Not using data augmentation.')
    model_1A.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model_1A.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        steps_per_epoch=x_train.shape[0] // batch_size,
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)

# Load label names to use in prediction results
label_list_path = 'datasets/cifar-10-batches-py/batches.meta'


keras_dir = os.path.expanduser(os.path.join('~', '.keras'))
datadir_base = os.path.expanduser(keras_dir)
if not os.access(datadir_base, os.W_OK):
    datadir_base = os.path.join('/tmp', '.keras')
label_list_path = os.path.join(datadir_base, label_list_path)

with open(label_list_path, mode='rb') as f:
    labels = pickle.load(f)

# Evaluate model with test data set and share sample prediction results
evaluation = model_1A.evaluate_generator(datagen.flow(x_test, y_test,
                                                   batch_size=batch_size,
                                                   shuffle=False),
                                      steps=x_test.shape[0] // batch_size,
                                      workers=4)
print('Model Accuracy = %.2f' % (evaluation[1]))

predict_gen = model_1A.predict_generator(datagen.flow(x_test, y_test,
                                                   batch_size=batch_size,
                                                   shuffle=False),
                                      steps=x_test.shape[0] // batch_size,
                                      workers=4)

for predict_index, predicted_y in enumerate(predict_gen):
    actual_label = labels['label_names'][np.argmax(y_test[predict_index])]
    predicted_label = labels['label_names'][np.argmax(predicted_y)]
    print('Actual Label = %s vs. Predicted Label = %s' % (actual_label,
                                                          predicted_label))
    if predict_index == num_predictions:
        break

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
Using real-time data augmentation.
Epoch 1/22
1562/1562 [==============================] - 105s - loss: 1.7554 - acc: 0.3684 - val_loss: 1.5188 - val_acc: 0.4671
Epoch 2/22
1562/1562 [==============================] - 102s - loss: 1.5200 - acc: 0.4593 - val_loss: 1.3490 - val_acc: 0.5165
Epoch 3/22
1562/1562 [==============================] - 104s - loss: 1.3835 - acc: 0.5095 - val_loss: 1.2315 - val_acc: 0.5723
Epoch 4/22
1562/1562 [==============================] - 103s - loss: 1.3082 - acc: 0.5418 - val_loss: 1.2024 - val_acc: 0.5759
Epoch 5/22
1562/1562 [==============================] - 102s - loss: 1.2615 - acc: 0.5604 - val_loss: 1.1082 - val_acc: 0.6134
Epoch 6/22
1562/1562 [==============================] - 102s - loss: 1.2280 - acc: 0.5736 - val_loss: 1.0907 - val_acc: 0.6229
Epoch 7/22
1562/1562 [==============================] - 101s - loss: 1.1995 - acc: 0.5851 - val_loss: 1.0872 - val_acc: 0.6169
Epo

In [13]:
#Configration 3

#Using Tanh Activation Function 

from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D

import os
import pickle
import numpy as np

batch_size = 32
num_classes = 10
epochs = 22
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model_1A.h5'

# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model_1A = Sequential()

model_1A.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model_1A.add(Activation('tanh'))
model_1A.add(Conv2D(32, (3, 3)))
model_1A.add(Activation('tanh'))
model_1A.add(MaxPooling2D(pool_size=(2, 2)))
model_1A.add(Dropout(0.25))

model_1A.add(Conv2D(64, (3, 3), padding='same'))
model_1A.add(Activation('tanh'))
model_1A.add(Conv2D(64, (3, 3)))
model_1A.add(Activation('tanh'))
model_1A.add(MaxPooling2D(pool_size=(2, 2)))
model_1A.add(Dropout(0.25))

model_1A.add(Flatten())
model_1A.add(Dense(512))
model_1A.add(Activation('tanh'))
model_1A.add(Dropout(0.5))
model_1A.add(Dense(num_classes))
model_1A.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model_1A.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

if not data_augmentation:
    print('Not using data augmentation.')
    model_1A.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model_1A.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        steps_per_epoch=x_train.shape[0] // batch_size,
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)

# Load label names to use in prediction results
label_list_path = 'datasets/cifar-10-batches-py/batches.meta'


keras_dir = os.path.expanduser(os.path.join('~', '.keras'))
datadir_base = os.path.expanduser(keras_dir)
if not os.access(datadir_base, os.W_OK):
    datadir_base = os.path.join('/tmp', '.keras')
label_list_path = os.path.join(datadir_base, label_list_path)

with open(label_list_path, mode='rb') as f:
    labels = pickle.load(f)

# Evaluate model with test data set and share sample prediction results
evaluation = model_1A.evaluate_generator(datagen.flow(x_test, y_test,
                                                   batch_size=batch_size,
                                                   shuffle=False),
                                      steps=x_test.shape[0] // batch_size,
                                      workers=4)
print('Model Accuracy = %.2f' % (evaluation[1]))

predict_gen = model_1A.predict_generator(datagen.flow(x_test, y_test,
                                                   batch_size=batch_size,
                                                   shuffle=False),
                                      steps=x_test.shape[0] // batch_size,
                                      workers=4)

for predict_index, predicted_y in enumerate(predict_gen):
    actual_label = labels['label_names'][np.argmax(y_test[predict_index])]
    predicted_label = labels['label_names'][np.argmax(predicted_y)]
    print('Actual Label = %s vs. Predicted Label = %s' % (actual_label,
                                                          predicted_label))
    if predict_index == num_predictions:
        break

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
Using real-time data augmentation.
Epoch 1/22
1562/1562 [==============================] - 97s - loss: 1.7408 - acc: 0.3745 - val_loss: 1.5062 - val_acc: 0.4562
Epoch 2/22
1562/1562 [==============================] - 94s - loss: 1.4935 - acc: 0.4713 - val_loss: 1.2814 - val_acc: 0.5509
Epoch 3/22
1562/1562 [==============================] - 94s - loss: 1.3761 - acc: 0.5140 - val_loss: 1.2519 - val_acc: 0.5611
Epoch 4/22
1562/1562 [==============================] - 94s - loss: 1.2949 - acc: 0.5468 - val_loss: 1.1607 - val_acc: 0.5887
Epoch 5/22
1562/1562 [==============================] - 94s - loss: 1.2519 - acc: 0.5640 - val_loss: 1.1096 - val_acc: 0.6190
Epoch 6/22
1562/1562 [==============================] - 94s - loss: 1.2214 - acc: 0.5755 - val_loss: 1.0762 - val_acc: 0.6265
Epoch 7/22
1562/1562 [==============================] - 93s - loss: 1.1931 - acc: 0.5873 - val_loss: 1.0733 - val_acc: 0.6279
Epoch 8/22

In [14]:
#Configration 4
#Using elu with maxpooling

from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D

import os
import pickle
import numpy as np

batch_size = 32
num_classes = 10
epochs = 22
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model_1A.h5'

# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model_1A = Sequential()

model_1A.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model_1A.add(Activation('elu'))
model_1A.add(Conv2D(32, (3, 3)))
model_1A.add(Activation('elu'))
model_1A.add(MaxPooling2D(pool_size=(2, 2)))
model_1A.add(Dropout(0.25))

model_1A.add(Conv2D(64, (3, 3), padding='same'))
model_1A.add(Activation('elu'))
model_1A.add(Conv2D(64, (3, 3)))
model_1A.add(Activation('elu'))
model_1A.add(MaxPooling2D(pool_size=(2, 2)))
model_1A.add(Dropout(0.25))

model_1A.add(Flatten())
model_1A.add(Dense(512))
model_1A.add(Activation('elu'))
model_1A.add(Dropout(0.5))
model_1A.add(Dense(num_classes))
model_1A.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model_1A.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

if not data_augmentation:
    print('Not using data augmentation.')
    model_1A.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model_1A.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        steps_per_epoch=x_train.shape[0] // batch_size,
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)

# Load label names to use in prediction results
label_list_path = 'datasets/cifar-10-batches-py/batches.meta'


keras_dir = os.path.expanduser(os.path.join('~', '.keras'))
datadir_base = os.path.expanduser(keras_dir)
if not os.access(datadir_base, os.W_OK):
    datadir_base = os.path.join('/tmp', '.keras')
label_list_path = os.path.join(datadir_base, label_list_path)

with open(label_list_path, mode='rb') as f:
    labels = pickle.load(f)

# Evaluate model with test data set and share sample prediction results
evaluation = model_1A.evaluate_generator(datagen.flow(x_test, y_test,
                                                   batch_size=batch_size,
                                                   shuffle=False),
                                      steps=x_test.shape[0] // batch_size,
                                      workers=4)
print('Model Accuracy = %.2f' % (evaluation[1]))

predict_gen = model_1A.predict_generator(datagen.flow(x_test, y_test,
                                                   batch_size=batch_size,
                                                   shuffle=False),
                                      steps=x_test.shape[0] // batch_size,
                                      workers=4)

for predict_index, predicted_y in enumerate(predict_gen):
    actual_label = labels['label_names'][np.argmax(y_test[predict_index])]
    predicted_label = labels['label_names'][np.argmax(predicted_y)]
    print('Actual Label = %s vs. Predicted Label = %s' % (actual_label,
                                                          predicted_label))
    if predict_index == num_predictions:
        break

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
Using real-time data augmentation.
Epoch 1/22
1562/1562 [==============================] - 98s - loss: 1.7388 - acc: 0.3746 - val_loss: 1.4903 - val_acc: 0.4627
Epoch 2/22
1562/1562 [==============================] - 102s - loss: 1.4786 - acc: 0.4754 - val_loss: 1.2765 - val_acc: 0.5576
Epoch 3/22
1562/1562 [==============================] - 101s - loss: 1.3625 - acc: 0.5193 - val_loss: 1.2545 - val_acc: 0.5510
Epoch 4/22
1562/1562 [==============================] - 102s - loss: 1.2869 - acc: 0.5499 - val_loss: 1.1229 - val_acc: 0.6101
Epoch 5/22
1562/1562 [==============================] - 101s - loss: 1.2376 - acc: 0.5693 - val_loss: 1.1660 - val_acc: 0.5925
Epoch 6/22
1562/1562 [==============================] - 101s - loss: 1.2061 - acc: 0.5806 - val_loss: 1.0943 - val_acc: 0.6097
Epoch 7/22
1562/1562 [==============================] - 98s - loss: 1.1742 - acc: 0.5919 - val_loss: 1.0531 - val_acc: 0.6320
Epoch

In [16]:
#Configration 5
#Maxpooling with leakyrelu

from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, advanced_activations
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D

import os
import pickle
import numpy as np

batch_size = 32
num_classes = 10
epochs = 22
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model_1A.h5'

# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model_1A = Sequential()

model_1A.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model_1A.add(advanced_activations.LeakyReLU(alpha=0.3))
model_1A.add(Conv2D(32, (3, 3)))
model_1A.add(advanced_activations.LeakyReLU(alpha=0.3))
model_1A.add(MaxPooling2D(pool_size=(2, 2)))
model_1A.add(Dropout(0.25))

model_1A.add(Conv2D(64, (3, 3), padding='same'))
model_1A.add(advanced_activations.LeakyReLU(alpha=0.3))
model_1A.add(Conv2D(64, (3, 3)))
model_1A.add(advanced_activations.LeakyReLU(alpha=0.3))
model_1A.add(MaxPooling2D(pool_size=(2, 2)))
model_1A.add(Dropout(0.25))

model_1A.add(Flatten())
model_1A.add(Dense(512))
model_1A.add(advanced_activations.LeakyReLU(alpha=0.3))
model_1A.add(Dropout(0.5))
model_1A.add(Dense(num_classes))
model_1A.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model_1A.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

if not data_augmentation:
    print('Not using data augmentation.')
    model_1A.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model_1A.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        steps_per_epoch=x_train.shape[0] // batch_size,
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)

# Load label names to use in prediction results
label_list_path = 'datasets/cifar-10-batches-py/batches.meta'


keras_dir = os.path.expanduser(os.path.join('~', '.keras'))
datadir_base = os.path.expanduser(keras_dir)
if not os.access(datadir_base, os.W_OK):
    datadir_base = os.path.join('/tmp', '.keras')
label_list_path = os.path.join(datadir_base, label_list_path)

with open(label_list_path, mode='rb') as f:
    labels = pickle.load(f)

# Evaluate model with test data set and share sample prediction results
evaluation = model_1A.evaluate_generator(datagen.flow(x_test, y_test,
                                                   batch_size=batch_size,
                                                   shuffle=False),
                                      steps=x_test.shape[0] // batch_size,
                                      workers=4)
print('Model Accuracy = %.2f' % (evaluation[1]))

predict_gen = model_1A.predict_generator(datagen.flow(x_test, y_test,
                                                   batch_size=batch_size,
                                                   shuffle=False),
                                      steps=x_test.shape[0] // batch_size,
                                      workers=4)

for predict_index, predicted_y in enumerate(predict_gen):
    actual_label = labels['label_names'][np.argmax(y_test[predict_index])]
    predicted_label = labels['label_names'][np.argmax(predicted_y)]
    print('Actual Label = %s vs. Predicted Label = %s' % (actual_label,
                                                          predicted_label))
    if predict_index == num_predictions:
        break

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
Using real-time data augmentation.
Epoch 1/22
1562/1562 [==============================] - 122s - loss: 1.8069 - acc: 0.3419 - val_loss: 1.5099 - val_acc: 0.4600
Epoch 2/22
1562/1562 [==============================] - 124s - loss: 1.5274 - acc: 0.4461 - val_loss: 1.3228 - val_acc: 0.5392
Epoch 3/22
1562/1562 [==============================] - 123s - loss: 1.4048 - acc: 0.4972 - val_loss: 1.2489 - val_acc: 0.5633
Epoch 4/22
1562/1562 [==============================] - 124s - loss: 1.3119 - acc: 0.5359 - val_loss: 1.1365 - val_acc: 0.5974
Epoch 5/22
1562/1562 [==============================] - 123s - loss: 1.2387 - acc: 0.5615 - val_loss: 1.0562 - val_acc: 0.6316
Epoch 6/22
1562/1562 [==============================] - 128s - loss: 1.1713 - acc: 0.5889 - val_loss: 0.9992 - val_acc: 0.6496
Epoch 7/22
1562/1562 [==============================] - 120s - loss: 1.1168 - acc: 0.6069 - val_loss: 0.9461 - val_acc: 0.6719
Epo